<img align="right" src="https://docs.google.com/uc?id=1tSEwB0qGJQatNAnntC3jVpuRQ1qV5TU8" width="200">

# Introduction to Time Series 

Course Instructor: Nicole LEE ([LinkedIn](https://www.linkedin.com/in/nicoleleezhiying/))

30 Mar 2023 (0900 - 1730)

***

### Course Overview

Time series analysis is a statistical method that deals with data collected over time. It involves analysing and modelling data to identify patterns and trends, and make predictions about future values in the series. Time series data can be found in a wide range of fields, including economics, finance, and engineering: 

- [IoT Sensor Predictions](https://www.timescale.com/blog/how-everactive-powers-a-dense-sensor-network-with-virtually-no-power-at-all/)
- Predictions of Financial Markets
- Prediction of macroeconomic factors which may contribute to business outcomes
- Understand sales patterns and predict future sales and revenue

<img align="center" src="https://www.timescale.com/blog/content/images/2023/03/time-series-data_bitcoin_img2.png" width="600">

In this 1-day course, you will learn about the basics of time series, data preparation of a time series dataset, visualization, modeling and forecasting to understand the underlying factors driving trends and provide impactful insights.

***
### Learning Objectives

1. [Introduction to Time Series](#sec_intro) <br>
   1.1 [Stationarity and Tests for Stationarity](#subsec_stationarity) <br>
   1.2 [Seasonality and Correlatation](#subsec_seasonality)<br>
2. [Univariate Time Series Forecasting](#sec_univ)<br>
   2.1 [Evaluation Metrics](#subsec_evaluation)<br>
   2.2 [Moving Average](#subsec_ma)<br>
   2.3 [Simple Exponential Smoothing, Holt's ES, Holt-Winter's ES](#subsec_es)<br>
   2.4 [AR, MA, ARIMA, SARIMA, SARIMAX Models](#subsec_arima)<br>
   2.4.1 [Summary](#subsec_uv_summary)<br>
   2.6 [Fourier Transformations](#subsec_FFT)<br>
3. [Multivariate Time Series Forecasting: VAR/VECM](#sec_multiv)<br>
4. [Group Project](#sec_proj)<br>

<!-- 1. [Basics](#sec_basics) <br> 
    1.1 [What is Folium, and what can it be used for?](#subsec_folium) <br>
    1.2 [Plotting markers, circles and circle markers](#subsec_markers) <br> 
    1.3 [Adding images](#subsec_images) <br>
    1.4 [Adding convenient tools](#subsec_tools) <br> 
<br>
2. [S$1 Million Dollar Resale Flats in SG](#sec_resale) <br> 
    2.1 [Getting coordinates of flats from onemap API](#subsec_flatcoords) <br>
    2.2 [Plotting flat locations using ClusterMap](#subsec_flatloc) <br>
    2.3 [Plotting boundaries](#subsec_boundaries) <br>
    2.4 [Plotting choropleths](#subsec_choropleths) <br>
<br>
3. [Timestamped Path](#sec_path) <br> -->

***

### Other Exercises to Try

1. Stock Price Prediction
   - Pick a stock of interest
   - Get price data from [Yahoo! Finance](https://sg.finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC)
   - Identify characteristics of the time series
   - Plot short-term predictions of stock prices

2. HDB Pricing Trend
   - Download HDB Data from [data.gov.sg](https://data.gov.sg/dataset/resale-flat-prices)
   - Consider breaking data points into its town boundaries, flat type, storey range, proximity to a facility, etc. and investigate trends in housing prices.
    
3. Find datasets on UCI/Kaggle and practice

***
<a id='sec_intro'></a>
### 1. Introduction to Time Series

Time series is when a set of data points are chronologically collected over time. These allow us to identify changes of different types of data points over time. 

***
<img align="center" src="http://www.weather.gov.sg/wp-content/uploads/2020/03/Fig1.png" width="600">


<u>Weather</u><br>
For instance, we notice that the average annual temperatures have risen over the last 80 years. We can also overlay the rainfall dataset to compare the average rainfall per month over years, to study if there has been a change in the <i>monsoon season</i> over the past 5 years.<br>
<img align="center" src="images/Monthly_Total_Rain.png" width="600"><br>


<u>Stock Market</u><br>
<img align="center" src="images/DJI_YahooFinanceChart.png" width="600"><br>
Often, we see the stock prices rise and fall, reacting to various economic events. By stripping away the <i>noise</i> of the data, we will be able to gain a better understanding of the price trends or volatility trends. However, due to the various interactions between market participants, as well as concurrent micro and macro economic factors, it is often not possible to strip the stock prices data and understand the true drivers of a price surge or dip in prices. 


<u>Sale and Transaction Data</u><br>
We will notice spikes in sales during a sale period. When you imagine a traditional retail market, you notice that the holiday period sees a peak, resulting in a clear yearly peak.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.tsa.api as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
## Download and understand the components of a time series dataset
rain_data = pd.read_csv("timeseries_sample.csv")
rain_data.head(10)

***
### Characteristics of a Time Series
A time series dataset is characterised by the following: <br>
- Trend: A trend is a long-term pattern in the data that shows a consistent increase or decrease over time. Trends can be linear (or additive) or nonlinear (or multiplicative), and can be observed at different time scales.
- Seasonality: Seasonality refers to regular, predictable patterns that repeat over a fixed time interval, such as daily, weekly, monthly or yearly. For example, sales of ice cream typically increase during summer months and decrease during winter months.
- Cyclical: Cyclical patterns are similar to seasonality but occur over a longer time period, often several years. These patterns are often related to economic or business cycles.
- Noise (or irregularity): Refers to fluctuations or randomness in the data that are not explained by trend, seasonality, or cyclical patterns. These 

<img align="center" src="https://otexts.com/fpp2/fpp_files/figure-html/fourexamples-1.png" width="600"><br>
These form the components of a time series.<br>
<br>
<b><u>Other terms which may be relevant include</u></b>:<br> 
<u>Lag</u><br>
Refers to the time difference between two related variables or time delay between a dependent and independent variable. By understanding lag, it helps us understand how one unit of change in one variable can affect another variable over time.
By studying the relationship of lag and its variables, we can identify the data trends across time.

***
<a id='subsec_stationarity'></a>

#### 1.1 Stationarity and Tests for Stationarity
Any time series dataset can be classified as stationary or non-stationarity, <br>
- Stationary data refers to data where the statistical properties, such as mean and variance, remain constant over time. This means that the distribution of data points is consistent across all time periods, and there are no trends or patterns that change over time. Therefore, stationary time series or stationary processes are predictable. Analytical methodologies can be applied across all time periods in the data set. <br>
- Non-stationary data refers to data whose statistical properties change over time. Hence, the trends, patterns or seasonality may differ and be difficult to predict over time. <br>
<br>
<br>

Stationarity can be further broken down into the following: <br>
- <b>Strong/Strict/Complete Stationarity</b>: A process is <b>invariant</b> under a shift in time.<br>
   This means all statistical properties are constant throughout time. 
- <b>Weak/Second-order/Covariance Stationarity</b>: A processes's first and second moments are invariant under a shift in time. <br>
   So, the <u>mean</u> and <u>autocovariance</u> of a series are constant over time, but its variance may still change. <br>
<br>
<br>

Time series methodologies in this day course are most effective on stationarity data, which can be tested in 2 ways:<br>
1. Visualization<br>
2. Augmented Dickey Fuller Test<br>

***
<i>Mathematically, this is represented as: <br>
1. Strong/Strict/Complete Stationarity<br>
   Let us label a set of data points as 
   $$\{X_t: t\in T\}$$
   where $t$ is the index, or taken as time here and $T$ is the index set, which is the total time period we consider. $X$ can be univariate or multivariate, continuous or discrete. 

   Now, $\{X_t\}$ is said to be completely stationary, if for all $n \geq 1$, for any $t_1, t_2, \dots, t_n \in T$ and for any lag $\tau$ such that $t_1+\tau, t_2+\tau, \dots, t_n+\tau \in T$ are also contained in the index set, the joint cdf $\{X_{t_1}, X_{t_2}, \dots, X_{t_n}\}$ is the same as that of $\{X_{t_{1+\tau}}, X_{t_{2+\tau}}, \dots, X_{t_{n+\tau}}\}$ Then, 
   $$ F_{t_1, t_2, \dots, t_n}(a_1, a_2, \dots, a_n) = F_{t_(1+\tau), t_(2+\tau), \dots, t_(n+\tau)}(a_1, a_2, \dots, a_n)$$
   <br>
   <b>A completely stationary process is invariant under a shift in time.</b><br>
   <br>
   <br>
   <br>
2. Weak/second-order Stationarity<br>
   Now, $\{X_t\}$ is said to be second-order stationary, if for all $n \geq 1$, for any $t_1, t_2, \dots, t_n \in T$ and for any lag $\tau$ such that $t_1+\tau, t_2+\tau, \dots, t_n+\tau \in T$ are also contained in the index set, all the joint moments of orders 1 and 2 of $\{X_{t_1}, X_{t_2}, \dots, X_{t_n}\}$ exist, are finite, and equal to the corresponding joint moments of $\{X_{t_{1+\tau}}, X_{t_{2+\tau}}, \dots, X_{t_{n+\tau}}\}$ Then, 
   $$ E\{X_t\} \equiv \mu, \qquad var\{X_t\} \equiv \sigma^2 = E\{X_t^2\} - \mu^2$$
   are constants independent of t. </i>
***

In [ ]:
### All time series data should have a time-index. Clean the dataset such that it has one
value_col = "Daily Rainfall Total (mm)"
# rain_data = ...

In [ ]:
### Consider the type of data and the appropriate size of a rolling window
### Consider trying a few values to determine if the dataset is indeed stationary
## Visual Stationarity Test
rolling_window = 12
rmean = rain_data[value_col].rolling(window=rolling_window).mean()
rstd = rain_data[value_col].rolling(window=rolling_window).std()

orig = plt.plot(rain_data[value_col], color="black", label="Original")
mean = plt.plot(rmean, color="red", label="Rolling Mean")
std = plt.plot(rstd, color="blue", label="Rolling Standard Deviation")
plt.legend(loc="best")
plt.title("Rolling mean and standard deviation")
plt.show(block=False)

In [ ]:
### Augmented Dickey Fuller Test
adftest = tsa.adfuller(rain_data[value_col], autolag="AIC")  # Autolag AIC vs BIC
dfoutput = pd.Series(
    adftest[0:4],
    index=["Test Statistic", "p-value", "#lags used", "number of observations used"],
)

for key, value in adftest[4].items():
    dfoutput["critical value (%s)" % key] = value
print(dfoutput)

If the series is non-stationary, often, it can be converted to a stationary series using a differencing method.
We call this first layer of differencing as first differences, where $\Delta Y_t = Y_t - Y_{t-1}$.<br>
<br>
$d$ = number of differencing steps it takes for a dataset to achieve stationarity.
Note that $d=0$ indicates a stationary series and $d>0$ indicates a non-stationary series

In [ ]:
### Form a stationary dataset from the sample

***
<a id='subsec_seasonality'></a>

#### 1.2 Seasonality

- Seasonality refers to regular, predictable patterns that repeat over a fixed time interval, such as daily, weekly, monthly or yearly.
- It can be caused by various factors, including seasons, holidays, weather, human behaviour, etc.
- Often seasonality has different frequencies and the actual seasonality predicted from the historical data may differ from what one's expertise about a subject.

<u>Methods to detect seasonality</u>
There are multiple methods of detecting seasonality, and/or cyclicity
- Visualize the data and not the periods of seasonality
- Use autocorrelation function (ACF) and partial auto-correlation function (PACF) plots to determine presence of seasonality
- Decompose time series into its trend, seasonality and residual components, allowing each component to be analysed individually

<u>ACF and PACF plots</u><br>
<br>
<b>Autocorrelation Function (ACF)</b>: Measures how a series is correlated with itself at different lags.<br>
<b>Partial Autocorrelation Function (PACF)</b> Regression of the series against its past lags. The terms can be interpreted as the effect of a change in the particular lag, while holding all other variables constant.<br>

In [ ]:
### Plot ACF and PACF plots and take note of the lags and decay, which helps identify the components of the time series

acf_pacf_axs = plt.subplots(2, 1)

plot_acf(rain_data[value_col], ax=acf_pacf_axs[0])
plot_pacf(rain_data[value_col], ax=acf_pacf_axs[1])
plt.show()

<u>Time Series Decomposition</u><br>
The time series can also be decomposed into its components, as stated above, i.e. its trend, seasonality and residuals. <br>
In addition, the models can be classified as additive or multiplicative models:
- Additive models: <br>
  $$Observation = trend + seasonality + residual$$
  The magnitude of the seasonal and residual components are independent of the trend.
- Multiplicative models
  $$ Observation = trend * seasonality * residual$$
  The multiplicative model has seasonal and/or residual plots which tend to follow the trend.
  They can be transformed into an additive model through a logarithmic transformation.
  $$ \therefore log(Observation) = log(time) + log(seasonality) + log(residual)
- Pseudo-additive models (<i>Extra</i>)
  Combines both additive and multiplicative models and are harder to spot. 

<i>Note:</i> Identifying the right model is important in finding the right method to remove seasonality.

In [ ]:
# TS Decomposition

decomposed_ts = tsa.seasonal_decompose(rain_data[value_col], freq=12)
white_noise = decomposed_ts.resid
print("Trend: {}".format(decomposed_ts.trend))
print("Seasonal: {}".format(decomposed_ts.seasonal))
print("White Noise: {}".format(white_noise))
print("Observed Values: {}".format(decomposed_ts.observed))

It is important to note that white noise has
- constant mean,
- constant variance,
- zero auto-correlation at all lags
Note how the auto-correlation of white noise have insignificant lags and lie within the confidence interval

For non-seasonal data, represented by $(p, d, q)$ 

|       | ACF                          | PACF                         |
|-------|------------------------------|------------------------------|
| AR(p) | Geometric Decay              | Significant till $p$ lags    |
| MA(q) | Significant till $q$ lags    | Geometric Decay              |

For seasonal data, represented by $(P, D, Q)m$
|       | ACF                          | PACF                         |
|-------|------------------------------|------------------------------|
| AR(p) | Geometric Decay              | Significant at $m$ lags      |
| MA(q) | Significant at $m$ lags      | Geometric Decay              |

***
<a id='sec_univ'></a>
### 2. Univariate Time Series Forecasting Methods

Given that we now know stationarity, and seasonality and the ways to detect then, we want to forecast the data based on its historical trends. 

We explore several statistical techniques in this section.

***
<a id='subsec_evaluation'></a>
#### 2.1 Evaluation Metrics

- MSE or Root MSE (RMSE)<br>
  $$\frac{1}{T} \sum_{t=1}^T (F_t - Y_t)^2, $$
  where $F_t$ is the forecasted value at time $t$ and $Y_t$ the actual value at time $t$.

- Mean absolute deviation (MAD)<br>
  $$\frac{1}{T}\sum_{t=1}^T |F_t - Y-t|,$$
  where $F_t$ is the forecasted value at time $t$ and $Y_t$ the actual value at time $t$.

- Mean absolute percentage error (MAPE)<br>
  $$\frac{1}{T}\sum_{t=1}^T |\frac{F_t - Y-t}{Y_t}|,$$
  where $F_t$ is the forecasted value at time $t$ and $Y_t$ the actual value at time $t$.

MAPE is unit free, and therefore ideal for comparing models.

*** 
<a id='subsec_ma'></a>

#### 2.2 Moving Average: MA(n)
Method used to smooth out fluctuations, also known as the "rolling average", therefore removing the effects of seasonality. 

The number of periods used to compute the moving/rolling average is known as the "window size" or "lag".


As n gets larger, the model is more robust and reflects the overall trend.

In [ ]:
### Use pd.DataFrame.rolling() to experiment with the moving averages

***
<a id='subsec_es'></a>
#### 2.3 Exponential Smoothing Methods (ES)
<u>Simple Exponential Smoothing (SES)</u><br>
$$ F_{t+1} = \alpha Y_t + (1 - \alpha)F_t,$$
where $\alpha$ is the smoothing parameter, smaller $\alpha$ = more sensitive to recent data changes.

- Gives more weight to recent observations, and less weight to older observations<br>
- Utilises a weighted average, instead of a simple average in MA<br>
- As events get older, the weights get exponentially smaller<br>
- Particularly useful when there is no clear seasonality<br>


<u> Holt's Exponential Smoothing (Holt's ES)</u><br>
$$ F_{t+p} = L_t + pT_t$$
$$ L_t = \alpha Y_t + (1 - \alpha)(L_{t-1} + T_{t-1})$$
$$ T_t = \beta(L_t - L_{t-1}) + (1 - \beta)T_{t-1},$$

where $F_{t+p}$ is a forecasted value for p time periods away, $L_t$ is the level (or intercept) for time $t$, and $T_t$ is the trend (or slope) at time $t$. <br>

- Extension of SES, taking into account the linear trend<br>
- <b>Two</b> smoothing parameters, one for level and other for trend<br>
- Particularly useful for data with a clear trend but no clear seasonality<br>


<u>Winter's Exponential Smoothing</u><br>
$$L_t = \alpha \frac{Y_t}{S_{t-s}} + (1-\alpha)(L_{t-1} + T_{t-1})$$
$$S_t = \gamma \frac{Y_t}{L_t} + (1 - \gamma) S_{t-s}$$
$$ T_t = \beta(L_t - L_{t-1}) + (1 - \beta)T_{t-1}$$
$$F_{t+p} = (L_t + pT_t)S_{t-s+p}$$
- Extends from Holt's ES, by adding a seasonal component.<br>
- <b>Three</b> smoothing parameters, $\alpha, \beta, \gamma$ for level, trend and seasonality respectively. <br>

In [ ]:
goog_train = pd.read_csv("goog_train.csv")
goog_test = pd.read_csv("goog_test.csv")

In [ ]:
### SES is suitable for data with no trend or seasonal pattern.
### forecast forward 100 steps with smoothing_level = 0.2


### Plot Original values and its forecasted values

In [ ]:
### SES should not be used on data with a trend or seasonal component.
### Practice differencing to remove trends. Consider using .diff()

In [ ]:
### Import various metrics from sklearn and experiment with them
from sklearn.metrics import mean_absolute_error

# MAD_SES = mean_absolute_error(
#     forecasted, actual
# )
# print("MAD of Simple Exponential Smoothing on differenced time series:", MAD_SES)

In [ ]:
### Try Holt's ES and forecast forward 100 steps.

In [ ]:
### Try your hand at Holt-Winter's ES and forecast forward 100 steps.
### Try using ExponentialSmoothing(train,trend='add', seasonal='add',seasonal_periods = 4).fit()

***
<a id='subsec_arima'></a>
#### 2.4 Models for Time Series datasets
Autoregressive Integrated Moving Average (ARIMA) - Linear models which can represent time series data (including non-stationary)<br>
ARIMA models do not involve independent variables and use information in the series to generate forecasts.<br>


<u>Autoregressive (AR) Model</u><br>
$AR(p)$: Uses previous $p$ (also called order of AR model) values to predict the future values
$$ Y_t = \phi_0 + \phi_1 Y_{t-1} + \dots + \phi_p Y_{t-p} + \epsilon_t,$$
where $\epsilon_t$ represents the white noise, $\{\phi_i\}_{i=1}^p$ are coefficients to estimate, and $Y_{i}$ are the values at time $i$
<i>Stationary time series can be represented by $AR(1)$</i><br>


<u>Moving Average (MA) Model</u><br>
$MA(q)$: Uses previous $p$ (also called order of AR model) error terms to predict the future values
$$ Y_t = \mu + \epsilon_t + \theta_1 \epsilon_{t-1} + \dots + \theta_q \epsilon_{t-q} \implies Y_t - \mu = \epsilon_t + \sum_{i = 1}^q \theta_i \epsilon_{t-i},$$
where $\epsilon_t$ represents the white noise, $\{\theta_i\}_{i=1}^q$ are coefficients to estimate, and $Y_{i}$ are the values at time $i$<br>
The deviation of the actual value at time t, is equal to the sum of current and past errors.


<i>Consider $MA(1) \equiv AR(\infty)$</i>

In [ ]:
### AR model
train_data = ...
test_data = ...

### Fit AR model
model_ar = tsa.AR(train_data)
model_ar_fit = model_ar.fit()

### Make Predictions
ar_predictions = model_ar_fit.predict(
    start=len(train_data), end=len(train_data) + len(test_data) - 1
)

print(
    "Mean Squared Error for a MA model is {}".format(
        mean_squared_error(test_data, ar_predictions)
    )
)

In [ ]:
### MA Model
model_ma = tsa.ARMA(train_data)
model_ma_fit = model_ma.fit()

ma_predictions = model_ma_fit.predict(
    start=len(train_data), end=len(train_data) + len(test_data) - 1
)

print(
    "Mean Squared Error for a MA model is {}".format(
        mean_squared_error(test_data, ma_predictions)
    )
)

<u>ARIMA Model</u><br>
<b>$ARIMA(p,d,q)$</b>: Combines AR and MA models, where 
  - $p$: Order of the AR model, 
  - $d$: Order of differencing (to make time series stationary),
  - $q$: Order of MA model


<u>SARIMA Model</u><br>
<b>$SARIMA(p,d,q)\times(P,D,Q)_s$</b>: Extension of ARIMA model which accounts for seasonality. AR and MA models, where 
  - $p$: Order of the AR model, 
  - $d$: Order of differencing (to make time series stationary),
  - $q$: Order of MA model
  - $P$: Seasonal order of AR model,
  - $D$: Seasonal order of differencing,
  - $Q$: Seasonal order of MA model
  - $s$: Seasonal period



***
Additionally, it would be good to know that the following models which account for external factors exist but will not be covered in today's session<br>
<u><i>ARIMAX Model</i></u>
<b>$ARIMAX(p,d,q,x)$</b>: "X" stands for Exogenous Variables, which extends the ARIMA model to account for the impact of external variables on the time series, where x represents the external variable.


<u><i>SARIMAX Model</i></u>
u>SARIMA Model</u><br>
<b>$SARIMA(p,d,q)\times(P,D,Q)_s$</b>: Extension of ARIMAX model which accounts for seasonality. AR and MA models, where 
  - $p$: Order of the AR model, 
  - $d$: Order of differencing (to make time series stationary),
  - $q$: Order of MA model
  - $P$: Seasonal order of AR model,
  - $D$: Seasonal order of differencing,
  - $Q$: Seasonal order of MA model
  - $s$: Seasonal period

In [ ]:
### ARIMA Model
model_arima = tsa.ARIMA()
model_arima_fit = model_arima.fit()

arima_predictions = model_arima_fit.predict(
    start=len(train_data), end=len(train_data) + len(test_data) - 1
)

print(
    "Mean Squared Error for a ARIMA model is {}".format(
        mean_squared_error(test_data, arima_predictions)
    )
)

In [ ]:
### SARIMA Model
model_sarima = tsa.SARIMA(train_data, order=..., seasonal_order=...)
model_sarima_fit = model_sarima.fit()

sarima_predictions = model_sarima_fit.predict(
    start=len(train_data), end=len(train_data) + len(test_data) - 1
)

print(
    "Mean Squared Error for a SARIMA model is {}".format(
        mean_squared_error(test_data, sarima_predictions)
    )
)

In [ ]:
### Can also consider installing pmdarima, which is the Python's version of R's auto.arima
# !pip install pmdarima

### from pmdarima install auto_arima

#### Mini Activity
1. Ingest the tractor sales dataset
2. Remove trends, for both mean and variance
3. Plot ACF and PACF to identify potential AR and MA model
4. Check `model.summary()`
5. Forecast sales using the best fit ARIMA model
6. Plot ACF and PACF for residuals of ARIMA model to ensure no more information is left for extraction

In [ ]:
### Code here

***
<a id='subsec_uv_summary'></a>
#### 2.4.1 Univariate Time Series Techniques Summary

1. Determine whether the series is stationary<br>
2. Plot ACF, PACF to determine seasonality<br>
3. Conduct differencing if non-stationary, iterate until stationarity is achieved<br>
4. Compare lags in ACF and PACF plots to identify the form of the model<br>
5. Use of evaluation metrics to evaluate the models - MSE, MAD, MAPE, Akaike's Information Criterion (AIC), Bayesian Information Criterion (BIC)<br>
      - Recall: Principle of parsimony in Data Science (BIC accounts for number of variables)<br>
6. Ensure residuals are random<br>
      - Ljung-Box Q Statistics: Plot ACF and PACF of the <b><u>residuals</u></b>. Approximately distributed as a $\chi$-square random variable with m-r degrees of freedom where r is the total number of parameters estimated in the ARIMA model<br>
7. Recall that forecasts further into the horizon are subjected to higher uncertainty. Forecasts should be recomputed as more data becomes available. This should be factored into the model monitoring process, which is not covered in today's session<br>


|Pros|Cons|
|--|---|
|Provide accurate short-range forecasts|A relatively large amount of data is required (> 50 for non-seasonal data and 6-10 years of seasonal data)|
Quite flexible and can represent a wide range of characteristics of time series|No easy way to update the parameters as new data become available|
Formal procedures for testing the adequacy of the model are available||
Forecasts and prediction intervals follow directly from the fitted model||

***
<a id='subsec_FFT'></a>
#### 2.5 Fourier Transformations (!Technical!)

- Fourier Transformations decomposes any function into sines and cosines (or waves)
- Widely used in time series analysis for understanding the periodicity and seasonality of a signal
- Converts time-domain to frequency-domain, which is obtained by decomposing it into a sum of sinusoidal functions of different frequencies, each of its own amplitude and phase. 


<u>Discrete Fourier Transform</u><br>
- Converts data to sequence of discrete frequencies.
- Use `np.fft` (Fast Fourier Transform) to compute the DFT. FFT is a faster version of the DFT from an algorithmic standpoint.
- The values at which the power spectrum plot peaks, demonstrates the strength of each frequency component. This translates loosely to the periodicity of a signal, or seasonality in time series.

In [ ]:
### Apply FFT and take absolute values
fft = np.abs(np.fft.fft(train_data))

### Compute the power spectrum
power = fft**2

### Compute the frequencies
freq = np.fft.fftfreq(len(power), d=1)  # assumes index is 1 unit apart here

# Plot the results
plt.plot(freq, power)
plt.show()

***
<a id='sec_multiv'></a>
### 3. Multivariate Time Series
Multivariate time series analysis, as its name suggests, has multiple time-dependent variables, i.e.the values of variable X at time $t$ may be dependent on variable $Y$ at time $t-1$. <br>
Vector Autoregression (VAR) and Vector Error Correction Model (VECM) are two popular techniques for analyzing multivariate time series data.<br>



<u>VAR Model</u><br>
Generalizes the univariate AR model, to accomodate multiple time series, modeled as a system of interdependent linear equations. <br>
Assumption: <br>
1. Each variable is stationary: Each variable is a function of its own lagged values, 
2. There exists a linear relationship between their current and past values<br>
Use of <i>Granger Causality tests</i> can determine if one variable can beused as a predictor of another variable.


<u>VECM Model</u><br>
Extends VAR to accomodate non-stationary time series data. <br>
Variables are first differenced to till stationary, and then the VAR model is run. <br>
Includes an error correction term, which captures the long-run equilibrium relationship among the variables.<br> 
Useful for data with cointegration, even if the variables are not directly causally related<br>


VAR and VECM models are estimated using AIC nad BIC criteria. (Lower AIC = Better Model)

In [8]:
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import select_coint_rank

### GC Test
print(grangercausalitytests(
    train_data[[:2]] 
    maxlag=15, addconst=True, verbose=True))

### VAR Model
VAR_model = tsa.VAR(endog=train_data)
VAR_results = VAR_model.fit(maxlags=2, ic='aic')

### Forecast using VAR model
lag_order = VAR_results.k_ar
forecast_input = train_data.values[-lag_order:]
var_fc = VAR_results.forecast(y=forecast_input, steps=len(test_data))

print(VAR_results.summary())

### Compute RMSE of VAR model
var_rmse = rmse(var_fc, test_data.values)

print(f"VAR RMSE: {var_rmse:.4f}")


SyntaxError: invalid syntax (<ipython-input-8-430eea106cd3>, line 6)

In [ ]:

vec_rank1 = vecm.select_coint_rank(
    train_data, det_order = 1, k_ar_diff = 1, method = 'trace', signif=0.01
    ) #methods = trace, maxeig

### VECM model
vecm_model = tsa.VECM(train_data, k_rank=1, deterministic='ci')
vecm_results = vecm_model.fit()~

### Forecast using VECM model
vecm_fc = vecm_results.predict(steps=len(test_data))

### Compute RMSE of VECM model
vecm_rmse = rmse(vecm_fc, test_data.values)


print(f"VECM RMSE: {vecm_rmse:.4f}")

***
<a id="sec_proj"></a>
### 4. Group Project
Consider one of the following datasets:<br>
- Beijing Multi-Site Air Quality Data,<br>
- Pasir Panjang Rainfall Data, (or other rainfall data from [Meteorological Service Singapore](http://www.weather.gov.sg/climate-historical-daily/)) <br>
- HDB Resale Prices <br>
- Stock data from Yahoo! Finance<br>

or any dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php?format=&task=&att=&area=&numAtt=10to100&numIns=&type=ts&sort=nameUp&view=table), [The UEA & UCR Time Series Classification Repository](https://www.timeseriesclassification.com/)

Using the time series dataset, conduct a time series decomposition and analysis of the dataset. 

At the end of the session, present your findings of the dataset.

In [ ]:
### Project Code Here

***
### Helpful Resources:
- [Forecasting: Principles and Practice](https://otexts.com/fpp2/)<br>
- [Jeffrey Yau: Time Series Forecasting using Statistical and Machine Learning Models](https://www.youtube.com/watch?v=_vQ0W_qXMxk)<br>
- [A Very Short Course on Time Series Analysis: Fourier Transform](https://bookdown.org/rdpeng/timeseriesbook/the-fourier-transform.html) <br>